In [3]:
import os

from openai import OpenAI

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key = OPENAI_API_KEY)

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo-0125',
    messages = [{'role':'user', 'content':'hi'}],
    temperature = 0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
import json
with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '서비스 및 위치가 매우 좋음', 'stars': 4, 'date': '3일 전'},
 {'review': '직원분들 친절하시고 잘 쉬다가 갑니다', 'stars': 5, 'date': '6일 전'},
 {'review': '다음에 또 올게요~', 'stars': 4, 'date': '2024.07.21'}]

In [5]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(175, 45)

In [6]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]


['[REVIEW_START]서비스 및 위치가 매우 좋음[REVIEW_END]',
 '[REVIEW_START]다음에 또 올게요~[REVIEW_END]',
 '[REVIEW_START]주변에서 노느라 숙소에 있는 시간은 짧았지만,,  조계사뷰라 놀다보니 새벽 5시쯤 절에서 종도 치고 독특한 경험이었어요ㅋ[REVIEW_END]']

In [7]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

In [8]:
import datetime
from dateutil import parser

def preprocess_reviews(path = './res/reviews.json'):
    with open('./res/reviews.json', 'r', encoding='utf-8') as f:
        review_list

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days = 6 * 30)

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except(ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad) 

    return reviews_good_text, reviews_bad_text

good = preprocess_reviews()
good

('[REVIEW_START]직원분들 친절하시고 잘 쉬다가 갑니다[REVIEW_END]\n[REVIEW_START]항상 오면 깔끔하고 쾌적한 호텔입니다[REVIEW_END]\n[REVIEW_START]더 좋아진것같습니다.[REVIEW_END]\n[REVIEW_START]깨끗하고 친절해서 좋았어요[REVIEW_END]\n[REVIEW_START]지리적 요건도 좋은것 같고 주차장 이용에도 편리했습니다.[REVIEW_END]\n[REVIEW_START]편하게 쉬다가 갑니당~~[REVIEW_END]\n[REVIEW_START]우선 접근성이 너무 좋구요(종로,광화문,경복궁,광장시장,인사동) 숙소도 청결하고 프론트 직원분들도 너무 친절하세요.. 건물에 식당도 있어서 너무나 편했어여..진짜 요기 추천이요..욕조 크게 필요하지 않으신분들은 미리 얘기하시면 욕조없는곳으로 배정해주시는데..전 없는쪽이 더 낫더라구요\r\n(사진이 용량이 너무 작아서 2장밖에 안올라가네여ㅜ)[REVIEW_END]\n[REVIEW_START]잘쉬었다 왔습니다 역시나 편하네요[REVIEW_END]\n[REVIEW_START]여자친구랑 편히 잘 쉬고 놀다 갑니당 방 깨끗하구 에어컨 잘 나와서 시원하네욤[REVIEW_END]\n[REVIEW_START]최고에요~~~~~~[REVIEW_END]\n[REVIEW_START]침대 너누 편해요. 단 주차비 내야하니 참조하세요[REVIEW_END]\n[REVIEW_START]위치가 최상이네요. 시설도 깔끔하고, 뷰도 좋았어요. 인사동쪽으로 간다면 최상의 선택!![REVIEW_END]\n[REVIEW_START]인사동 방문시에는 항상 이용하고 있습니다,.위치와 서비스 모두 만족합니다[REVIEW_END]\n[REVIEW_START]친절하고 위치가 좋습니다[REVIEW_END]\n[REVIEW_START]재방문 의사 1000%[REVIEW_END]\n[REVIEW_START]깨끗하고 너무 좋았어요[REVIEW_END]\n[REVIEW_START]위치도 아주좋고 숙소도

In [9]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

In [10]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘."""

In [11]:
reviews, _ = preprocess_reviews(path = './res/reviews.json')

def summarize(reviews, prompt, temperature = 0.0, model = 'gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model = model,
        messages = [{'role':'user', 'content': prompt}],
        temperature=temperature
    )
    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

직원 친절, 깨끗한 호텔, 좋은 위치와 주차 편리, 편안한 휴식, 최고의 숙소.


In [12]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [13]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A의 요약은 매우 간결하고 핵심적인 키워드만을 나열한 반면, Assistant B의 요약은 보다 상세하고 구체적입니다. 

1. **도움성**: Assistant B는 숙소의 위치, 청결, 직원의 친절함, 주변 편의시설 등 다양한 측면을 언급하여 사용자에게 더 많은 정보를 제공합니다.
2. **관련성**: 두 요약 모두 리뷰의 주요 포인트를 다루고 있지만, Assistant B는 더 많은 세부사항을 포함하여 리뷰의 내용을 더 충실히 반영합니다.
3. **정확성**: Assistant B는 리뷰에서 언급된 여러 요소를 정확하게 요약하고 있습니다. 예를 들어, 관광지 접근성, 청결, 직원 친절도 등.
4. **깊이**: Assistant B는 숙소의 여러 측면을 다루며 깊이 있는 요약을 제공합니다.
5. **창의성**: Assistant B는 단순한 키워드 나열이 아닌, 문장으로 구성된 요약을 제공하여 읽기 쉽고 이해하기 쉽게 만듭니다.

따라서, Assistant B의 요약이 더 나은 평가를 받을 만합니다.

[[B]]


In [14]:
eval_count = 10

summarise_baseline = [summarize(reviews, PROMPT_BASELINE, temperature = 1.0).choices[0].message.content for _ in range(eval_count)]
summarise_baseline

['직원 친절, 깔끔한 호텔, 좋아진 분위기, 친절하고 깨끗, 지리적 요건 용이.',
 '직원 친절, 깨끗한 숙소, 좋은 위치와 주변 시설, 재방문 의사 매우 높음, 편안하게 쉬었다 감 잡니다.',
 '직원 친절, 깨끗한 숙소, 좋은 위치와 주차편의, 편안한 숙박 경험을 강조하는 다양한 리뷰들이 있습니다.',
 '직원 친절, 깨끗하고 쾌적한 호텔, 위치 및 주차 편리, 휴식 취하기 좋음, 가격 대비 만족적인 숙소.',
 '직원 친절, 깔끔하고 쾌적한 숙소, 개선된 모습, 깨끗하고 친절한 서비스, 지리적 요건 우수함, 편안한 휴식 가능.',
 '직원 친절, 깨끗한 숙소, 좋은 위치, 편리한 주차장, 편안한 휴식 가능.',
 '직원 친절, 호텔 깔끔, 좋은 위치와 주차편리, 편안한 휴식 가능, 재방문 의사 있음',
 '직원 친절, 깨끗한 호텔, 좋은 위치와 주차 편리, 편안하고 휴식 가능, 아주 만족스러운 숙소 리뷰들을 받음.',
 '직원 친절, 깨끗한 호텔, 좋아진 모습, 친절하고 깨끗한 숙소, 지리적으로 편리함이 돋보임.',
 '직원 친절, 깨끗한 호텔, 좋아진 곳, 지리적 요건 우수, 편안한 숙박 경험']

In [15]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')
    
    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summarise_baseline, [summary_real_20240526 for _ in range(len(summarise_baseline))])
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")


100%|██████████| 10/10 [00:37<00:00,  3.74s/it]

Wins: 0, Losses: 10, Ties: 0


모델 고도화 1 - 조건들 명시

In [16]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:46<00:00,  4.67s/it]

Wins: 1, Losses: 9, Ties: 0


모델 고도화 2 - 입력 데이터의 품질 증가

In [17]:
import datetime
from dateutil import parser

def preprocess_reviews(path = './res/reviews.json'):
    with open('./res/reviews.json', 'r', encoding='utf-8') as f:
        review_list

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days = 6 * 30)

    filtered_cnt = 0

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except(ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue

        if len(r['review']) < 30:
            filtered_cnt += 1
            continue


        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    
    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_good[:min(len(reviews_bad), 50)]
        
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad) 

    print(len(reviews_good))

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

22


In [18]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:46<00:00,  4.69s/it]

Wins: 3, Losses: 7, Ties: 0


모델 고도화 3 - Few-Shot Prompting

In [21]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model = 'gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

22


100%|██████████| 10/10 [00:46<00:00,  4.61s/it]

Wins: 2, Losses: 8, Ties: 0


In [22]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:41<00:00,  4.11s/it]

Wins: 8, Losses: 2, Ties: 0


In [23]:
summaries

['이 숙소는 지리적으로 매우 우수한 위치에 자리하고 있으며, 주변의 인사동, 광화문, 경복궁 등 주요 관광지에 쉽게 접근할 수 있다는 평가를 받고 있습니다. 또한, 숙소의 청결도와 직원들의 친절함이 높은 만족도를 이끌어내고 있으며, 편의시설의 접근성도 좋다는 의견이 많습니다. 다양한 이용객들이 재방문을 원하며, 전체적으로 긍정적인 리뷰가 많은 것으로 보입니다. 방음에 대한 소란을 경험한 의견도 있지만, 숙박 경험을 즐기는 이용객들이 많아 전반적으로 좋은 성향을 보이고 있습니다.',
 '이 숙소는 지리적으로 매우 우수한 위치에 자리하고 있으며, 주변의 인사동, 광화문, 경복궁 등 주요 관광지에 쉽게 접근할 수 있다는 평가를 받고 있습니다. 숙소 내 청결도와 직원들의 친절함이 높게 평가되었고, 편의시설의 접근성도 좋다는 의견이 많이 나왔습니다. 재방문 의사를 밝힌 이용객들도 많아, 전반적으로 서비스 품질이 우수하다는 인상을 줍니다. 방음에 대한 아쉬움을 표현한 의견도 있지만, 대체로 숙소에 대한 긍정적인 리뷰가 많은 것으로 보입니다.',
 '숙소는 지리적으로 우수한 위치에 자리해 있어서 주변 관광지에 쉽게 접근할 수 있고, 청결도와 직원 서비스가 좋은 평가를 받고 있습니다. 재방문 의사가 높은 곳으로, 방음이 아쉬운 점은 있지만 대체로 긍정적인 리뷰가 많이 있습니다. 위치, 서비스, 시설 등 모든면에 대해 만족하며 다음 여행에도 묵고 싶은 곳으로 추천받고 있습니다.',
 '이 숙소는 지리적으로 매우 우수한 위치에 자리하고 있으며, 주변의 인사동, 광화문, 경복궁 등 주요 관광지에 쉽게 접근할 수 있다는 평가를 받고 있습니다. 또한, 숙소의 청결도와 직원들의 친절함이 높은 만족도를 이끌어내고 있으며, 편의시설의 접근성도 좋다는 의견이 많습니다. 여러 번 재방문할 의사를 밝힌 이용객들도 많아, 숙소의 전반적인 서비스 품질이 우수하다는 것을 알 수 있습니다. 방음에 대한 아쉬움을 표현한 소수의 의견도 있으나, 대체로 숙소에 대한 긍정적인 리뷰가 주를 이루고 있습니다.

In [24]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

22


In [25]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

22


100%|██████████| 10/10 [00:40<00:00,  4.08s/it]

Wins: 10, Losses: 0, Ties: 0
